In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt
bempp.core.opencl_kernels.set_default_device(0,0)

In [2]:
h = 0.2
wavenumber = 4

In [19]:
unit_sphere = bempp.api.shapes.sphere(r = 1, origin=(0,0,0), h = h)
vert_unit_sphere = unit_sphere.vertices
space_unit_sphere = bempp.api.function_space(unit_sphere, 'P', 1)

cube1 = bempp.api.shapes.sphere(r = 0.25, origin = (0.7,0,0), h = h)
space_cube1 = bempp.api.function_space(cube1,'P',1)

In [20]:
eta = wavenumber

identity1 = bempp.api.operators.boundary.sparse.identity(space_cube1, space_cube1, space_cube1)
slp1 = bempp.api.operators.boundary.helmholtz.single_layer(space_cube1, space_cube1, space_cube1, wavenumber)
dlp1 = bempp.api.operators.boundary.helmholtz.double_layer(space_cube1, space_cube1, space_cube1, wavenumber)
slp_far1 = helmholtz_farfield.single_layer(space_cube1, vert_unit_sphere, wavenumber)
dlp_far1 = helmholtz_farfield.double_layer(space_cube1, vert_unit_sphere, wavenumber)

lhs1 = 0.5 * identity1 + dlp1 - 1j* eta *slp1

In [6]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*wavenumber*r))*scipy.special.jv(q+0.5, wavenumber*r)


def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [7]:
d = [1,0,0]
def coeff_sph_expansion(p,q):
    """Analytical formula of the coefficients of the spherical expansions of plane waves"""
    coeff = 4*np.pi*(1j)**q*normalized_spherical_harmonics(-p,q,d)/(-1)**(abs(p))
    return coeff

In [26]:
deg = 10

sph_harm_coeff = []

for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def sph_harm(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)
        sph_harm_grid = bempp.api.GridFunction(space_unit_sphere, fun = sph_harm)
        sph_harm_coeff.append(np.conj(sph_harm_grid.coefficients)/((-1j)**(q+1)))

In [27]:
mass_mat = bempp.api.operators.boundary.sparse.identity(space_unit_sphere, space_unit_sphere, space_unit_sphere).weak_form().A

In [28]:
slp_far_field_coeff1 = []

for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def dirichlet_fun(x,n,domain_index,result):
                with objmode():
                    result[0] = - regular_spherical_wavefunctions(p,q,x)

        rhs_fun1 = bempp.api.GridFunction(space_cube1, fun = dirichlet_fun)
        field1, info1 = bempp.api.linalg.gmres(lhs1, rhs_fun1, tol=1E-5)
        slp_far_field_coeff1.append(dlp_far1.evaluate(field1) - 1j * eta * slp_far1.evaluate(field1))

In [29]:
T_matrix1 = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
for i in range((deg+1)**2):
    for j in range((deg+1)**2):
        T_matrix1[i,j] = (slp_far_field_coeff1[j] @ mass_mat @ sph_harm_coeff[i])*wavenumber

In [30]:
coeff_inc = []
for q in range(deg+1):
    for p in range(-q, q+1):
        coeff_inc.append(coeff_sph_expansion(p,q))

coeff_sca = T_matrix1 @ coeff_inc

In [31]:
def sca_far_field(x):
    fun = 0
    i = 0
    for q in range(deg+1):
        for p in range(-q,q+1):
            fun += coeff_sca[i]*(1/wavenumber)*(-1j)**(q+1) * normalized_spherical_harmonics(p, q, x)
            i +=1
    return fun

In [32]:
far_list_test = []
for i in range(vert_unit_sphere.shape[1]):
    far_list_test.append(sca_far_field(vert_unit_sphere[:,i]))

@bempp.api.complex_callable
def exact_data(x, n, domain_index, result):
    result[0] = - np.exp(1j * wavenumber * x[0])

grid_fun = bempp.api.GridFunction(space_cube1, fun=exact_data)
fun_exact, info_exact = bempp.api.linalg.gmres(lhs1, grid_fun, tol=1E-5)
res_exact = dlp_far1.evaluate(fun_exact) - 1j * eta * slp_far1.evaluate(fun_exact)
err = np.sqrt((far_list_test - res_exact[0,:]) @ mass_mat @ np.conj(far_list_test - res_exact[0,:]))/np.sqrt(res_exact[0,:]@ mass_mat @ np.conj(res_exact[0,:]))
print(err)

(0.016579177358242866-1.6318867910774145e-19j)


In [18]:
sca_far_field(vert_unit_sphere[:,1])

(-0.0618159009939167-0.4908272643574333j)

In [17]:
res_exact[0,1]

(-0.062431331544732044-0.49972714210290853j)

In [ ]:
sph_harm_coeff_test = []

for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def sph_harm(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)
        sph_harm_grid = bempp.api.GridFunction(space_unit_sphere, fun = sph_harm)
        sph_harm_coeff_test.append(np.conj(sph_harm_grid.coefficients)*((1j)**q))

In [ ]:
direction_sph_harm = []
for q in range(deg+1):
    for p in range(-q,q+1):
        direction_sph_harm.append(np.conj(normalized_spherical_harmonics(p,q,d)) * (1j**q))

In [ ]:
T_matrix1_test = np.zeros(((deg+1)**2,(deg+1)**2),dtype = complex)
for i in range((deg+1)**2):
    for j in range((deg+1)**2):
        if direction_sph_harm[j] == 0:
            T_matrix1_test[i,j] = 0
        else:
            T_matrix1_test[i,j] = (res_exact @ mass_mat @ sph_harm_coeff_test[i]) * 1j * wavenumber / (4*np.pi * direction_sph_harm[j])

In [ ]:
coeff_sca_test = T_matrix1_test @ coeff_inc

sca_far_field_test(vert_unit_sphere[:,0])

In [ ]:
T_matrix1_test - np.transpose(T_matrix1_test)

In [ ]:
T_matrix1 - np.transpose(T_matrix1)

In [ ]:
coeff_scattered_field = []
for q in range(deg+1):
    for p in range(-q,q+1):
        @bempp.api.complex_callable
        def sph_harm(x,n,domain_index,result):
            with objmode():
                result[0] = normalized_spherical_harmonics(p,q,x)
        sph_harm_grid = bempp.api.GridFunction(space_unit_sphere,fun = sph_harm)
        coeff_scattered_field.append(wavenumber* (1j**(q+1)) * (res_exact[0,:] @ mass_mat @ np.conj(sph_harm_grid.coefficients)))

In [ ]:
coeff_scattered_field - coeff_sca

In [ ]:
T_matrix1[3,3]

In [ ]:
T_matrix1[1,1]

In [ ]:
T_matrix1[0,1]

In [ ]:
T_matrix1[3,0]

In [ ]:
S_matrix = np.identity(T_matrix1.shape[0]) + 2*T_matrix1

print(np.linalg.norm(np.matrix(S_matrix).getH()@S_matrix  - np.identity(T_matrix1.shape[0])),
np.linalg.norm(np.matrix(S_matrix).getH()@S_matrix - S_matrix@np.matrix(S_matrix).getH()))